In [3]:
! pip install metagpt==0.6.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.3/271.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 32.

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_BASE_URL"] = userdata.get('AZURE_OPENAI_API_BASE')
os.environ["OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["DEPLOYMENT_NAME"] = userdata.get('fast_llm_model_deployment_id')
os.environ["DEPLOYMENT_ID"] = "gpt-35-turbo"
os.environ["CALC_USAGE"] = 'false'

## Example: 打印斐波那契数列

In [3]:
import asyncio
import re

from metagpt.actions.action import Action, ActionNode
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message


2024-01-21 05:54:19.100 | INFO     | metagpt.const:get_metagpt_package_root:32 - Package root set to /content
2024-01-21 05:54:19.252 | INFO     | metagpt.config:get_default_llm_provider_enum:124 - LLMProviderEnum.AZURE_OPENAI Model: gpt35turbo
2024-01-21 05:54:19.255 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.AZURE_OPENAI


In [4]:
# 将思考斐波那契数列的10个数字作为prompt输入，在这里我们将“思考需要生成的数字列表”作为命令（instruction）写入
 # 将期望返回格式（expected_type）设置为str，无需设置例子（example）
SIMPLE_THINK_NODE = ActionNode(
    key="Simple Think Node",
    expected_type=str,
    instruction="""
            Think about what list of numbers you need to generate
            """,
    example=""
)

# 在这里通过命令（instruction）来规定需要生成的数字列表格式，提供例子（example）来帮助LLM理解
SIMPLE_CHECK_NODE = ActionNode(
    key="Simple CHECK Node",
    expected_type=str,
    instruction="""
            Please provide the number list for me, strictly following the following requirements:
            1. Answer strictly in the list format like [1,2,3,4]
            2. Do not have extra spaces or line breaks.
            Return the list here:
            """,
    example="[1,2,3,4]"
            "[4,5,6]",
 )


### ActionNode的子类

In [5]:
class THINK_NODES(ActionNode):
    def __init__(self, name="Think Nodes", expected_type=str, instruction="", example=""):
        super().__init__(key=name, expected_type=str, instruction=instruction, example=example)
        self.add_children([SIMPLE_THINK_NODE, SIMPLE_CHECK_NODE])    # 初始化过程，将上面实现的两个子节点加入作为THINK_NODES类的子节点

    async def fill(self, context, llm, schema="raw", mode="auto", strgy="complex"):
        self.set_llm(llm)
        self.set_context(context)
        if self.schema:
            schema = self.schema

        if strgy == "simple":
            return await self.simple_fill(schema=schema, mode=mode)
        elif strgy == "complex":
            # 这里隐式假设了拥有children
            child_context = context    # 输入context作为第一个子节点的context
            for _, i in self.children.items():
                i.set_context(child_context)    # 为子节点设置context
                child = await i.simple_fill(schema=schema, mode=mode)
                child_context = child.content    # 将返回内容（child.content）作为下一个子节点的context

            self.content = child_context    # 最后一个子节点返回的内容设置为父节点返回内容（self.content）
            return self



### SimplePrint 动作和 Printer 角色

In [6]:
class SimplePrint(Action):
    """
    Action that print the num inputted
    """
    input_num: int = 0

    def __init__(self, name="SimplePrint", input_num:int=0):
        super().__init__()

        self.input_num = input_num

    async def run(self, **kwargs):
        print(str(self.input_num) + "\n")
        return 0

class ThinkAction(Action):
    """
    Action that think
    """

    def __init__(self, name="ThinkAction", context=None, llm=None):
        super().__init__()
        self.node = THINK_NODES()    # 初始化Action时，初始化一个THINK_NODE实例并赋值给self.node

    async def run(self, instruction) -> list:
        PROMPT = """
            You are now a number list generator, follow the instruction {instruction} and
            generate a number list to be printed please.
            """

        prompt = PROMPT.format(instruction=instruction)
        rsp_node = await self.node.fill(context=prompt, llm=self.llm, schema="raw",
                                        strgy="complex")  # 运行子节点，获取返回（返回格式为ActionNode）（注意设置 schema="raw" ）
        rsp = rsp_node.content  # 获取返回的文本内容

        rsp_match = self.find_in_brackets(rsp)  # 按列表格式解析返回的文本内容，定位“[”与“]”之间的内容

        try:
            rsp_list = list(map(int, rsp_match[0].split(',')))  # 按列表格式解析返回的文本内容，按“,”对内容进行分割，并形成一个python语法中的列表

            return rsp_list
        except:
            return []

    @staticmethod
    def find_in_brackets(s):
        pattern = r'\[(.*?)\]'
        match = re.findall(pattern, s)
        return match

In [7]:
class Printer(Role):

    def __init__(self, name="Jerry", profile="Printer", goal="Print the number", constraints=""):
        super().__init__()

        self._init_actions([ThinkAction])
        # self.num_list = list()

    async def _think(self) -> None:
        """Determine the action"""
        # logger.info(self._rc.state)

        if self.rc.todo is None:
            self._set_state(0)
            return

        if self.rc.state + 1 < len(self.states):
            self._set_state(self.rc.state + 1)
        else:
            self.rc.todo = None

    async def _prepare_print(self, num_list:list) -> Message:
        """Add actions"""
        actions = list()

        for num in num_list:
            actions.append(SimplePrint(input_num=num))

        self._init_actions(actions)
        self.rc.todo = None
        return Message(content=str(num_list))

    async def _act(self) -> Message:
        """Action"""
        todo = self.rc.todo

        if type(todo) is ThinkAction :
            msg = self.rc.memory.get(k=1)[0]
            self.goal = msg.content
            resp = await todo.run(instruction=self.goal)
            # logger.info(resp)

            return await self._prepare_print(resp)

        resp = await todo.run()
        # logger.info(resp)

        return Message(content=str(resp), role=self.profile)

    async def _react(self) -> Message:
        """"""
        while True:
            await self._think()

            if self.rc.todo is None:
                break
            msg = await self._act()

        return msg

In [8]:
async def main():
    msg = "Provide the first 10 numbers of the Fibonacci series"
    role = Printer()
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)


# if __name__ == '__main__':
    # asyncio.run(main())
await main()

2024-01-21 05:54:28.966 | INFO     | metagpt.config:get_default_llm_provider_enum:124 - LLMProviderEnum.AZURE_OPENAI Model: gpt35turbo
2024-01-21 05:54:28.969 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.AZURE_OPENAI
2024-01-21 05:54:29.060 | INFO     | metagpt.config:get_default_llm_provider_enum:124 - LLMProviderEnum.AZURE_OPENAI Model: gpt35turbo
2024-01-21 05:54:29.063 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.AZURE_OPENAI
2024-01-21 05:54:29.161 | INFO     | __main__:main:4 - Provide the first 10 numbers of the Fibonacci series


To generate the first 10 numbers of the Fibonacci series, we can start with 0 and 1, and then each subsequent number is the sum of the previous two numbers. Here is the number list:

0, 1, 1, 2, 3, 5, 8, 13, 21, 34

2024-01-21 05:54:30.695 | INFO     | metagpt.config:get_default_llm_provider_enum:124 - LLMProviderEnum.AZURE_OPENAI Model: gpt35turbo
2024-01-21 05:54:30.697 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.AZURE_OPENAI
2024-01-21 05:54:30.782 | INFO     | metagpt.config:get_default_llm_provider_enum:124 - LLMProviderEnum.AZURE_OPENAI Model: gpt35turbo
2024-01-21 05:54:30.784 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.AZURE_OPENAI


[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

2024-01-21 05:54:30.879 | INFO     | metagpt.config:get_default_llm_provider_enum:124 - LLMProviderEnum.AZURE_OPENAI Model: gpt35turbo
2024-01-21 05:54:30.882 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.AZURE_OPENAI
2024-01-21 05:54:30.972 | INFO     | metagpt.config:get_default_llm_provider_enum:124 - LLMProviderEnum.AZURE_OPENAI Model: gpt35turbo
2024-01-21 05:54:30.976 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.AZURE_OPENAI
2024-01-21 05:54:31.066 | INFO     | metagpt.config:get_default_llm_provider_enum:124 - LLMProviderEnum.AZURE_OPENAI Model: gpt35turbo
2024-01-21 05:54:31.069 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.AZURE_OPENAI
2024-01-21 05:54:31.189 | INFO     | metagpt.config:get_default_llm_provider_enum:124 - LLMProviderEnum.AZURE_OPENAI Model: gpt35turbo
2024-01-21 05:54:31.192 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: L

0

1

1

2

3

5

8

13

21

34



# 使用ActionNode实现一个Agent：技术文档助手

### ActionNode：重写WriteDirectory方法

In [17]:
import asyncio
import re

from metagpt.actions.action import Action, ActionNode
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message
from typing import Dict
from metagpt.utils.common import OutputParser

from metagpt.const import TUTORIAL_PATH
from metagpt.utils.file import File

from datetime import datetime


In [18]:
# 命令文本
DIRECTORY_STRUCTION = """
    You are now a seasoned technical professional in the field of the internet.
    We need you to write a technical tutorial".
    您现在是互联网领域的经验丰富的技术专业人员。
    我们需要您撰写一个技术教程。
    """

# 实例化一个ActionNode，输入对应的参数
DIRECTORY_WRITE = ActionNode(
    # ActionNode的名称
    key="DirectoryWrite",
    # 期望输出的格式
    expected_type=str,
    # 命令文本
    instruction=DIRECTORY_STRUCTION,
    # 例子输入，在这里我们可以留空
    example="",
 )

In [19]:


class WriteDirectory(Action):
    """Action class for writing tutorial directories.

    Args:
        name: The name of the action.
        language: The language to output, default is "Chinese".

        用于编写教程目录的动作类。
        参数：
        name：动作的名称。
        language：输出的语言，默认为"Chinese"。
    """

    language: str = "Chinese"

    def __init__(self, name: str = "", language: str = "Chinese", *args, **kwargs):
        super().__init__()
        self.language = language

    async def run(self, topic: str, *args, **kwargs) -> Dict:
        """Execute the action to generate a tutorial directory according to the topic.

        Args:
            topic: The tutorial topic.

        Returns:
            the tutorial directory information, including {"title": "xxx", "directory": [{"dir 1": ["sub dir 1", "sub dir 2"]}]}.
        根据主题执行生成教程目录的操作。
            参数：
            topic：教程主题。
            返回：
            教程目录信息，包括{"title": "xxx", "directory": [{"dir 1": ["sub dir 1", "sub dir 2"]}]}.
        """

        DIRECTORY_PROMPT = """
        The topic of tutorial is {topic}. Please provide the specific table of contents for this tutorial, strictly following the following requirements:
        1. The output must be strictly in the specified language, {language}.
        2. Answer strictly in the dictionary format like {{"title": "xxx", "directory": [{{"dir 1": ["sub dir 1", "sub dir 2"]}}, {{"dir 2": ["sub dir 3", "sub dir 4"]}}]}}.
        3. The directory should be as specific and sufficient as possible, with a primary and secondary directory.The secondary directory is in the array.
        4. Do not have extra spaces or line breaks.
        5. Each directory title has practical significance.
        教程的主题是{topic}。请按照以下要求提供本教程的具体目录：
        1. 输出必须严格符合指定语言，{language}。
        2. 回答必须严格按照字典格式，如{{"title": "xxx", "directory": [{{"dir 1": ["sub dir 1", "sub dir 2"]}}, {{"dir 2": ["sub dir 3", "sub dir 4"]}}]}}。
        3. 目录应尽可能具体和充分，包括一级和二级目录。二级目录在数组中。
        4. 不要有额外的空格或换行符。
        5. 每个目录标题都具有实际意义。
        """

        # 我们设置好prompt，作为ActionNode的输入
        prompt = DIRECTORY_PROMPT.format(topic=topic, language=self.language)
        # resp = await self._aask(prompt=prompt)
        # 直接调用ActionNode.fill方法，注意输入llm
        # 该方法会返回self，也就是一个ActionNode对象
        print("prompt: ", prompt)
        resp_node = await DIRECTORY_WRITE.fill(context=prompt, llm=self.llm, schema="raw")
        # 选取ActionNode.content，获得我们期望的返回信息
        resp = resp_node.content
        return OutputParser.extract_struct(resp, dict)

In [20]:
class WriteContent(Action):
    """Action class for writing tutorial content.

    Args:
        name: The name of the action.
        directory: The content to write.
        language: The language to output, default is "Chinese".
    """

    language: str = "Chinese"
    directory: str = ""
    total_content: str = "" ## 组装所有子节点的输出

    def __init__(self, name: str = "", action_nodes: list = [], language: str = "Chinese", *args, **kwargs):
        super().__init__()
        self.language = language
        self.node = ActionNode.from_children("WRITE_CONTENT_NODES", action_nodes) ## 根据传入的action_nodes列表，生成一个父节点

    async def run(self, topic: str, *args, **kwargs) -> str:
        COMMON_PROMPT = """
        You are now a seasoned technical professional in the field of the internet.
        We need you to write a technical tutorial with the topic "{topic}".
        """
        CONTENT_PROMPT = COMMON_PROMPT + """
        Now I will give you the module directory titles for the topic.
        Please output the detailed principle content of this title in detail.
        If there are code examples, please provide them according to standard code specifications.
        Without a code example, it is not necessary.

        The module directory titles for the topic is as follows:
        {directory}

        Strictly limit output according to the following requirements:
        1. Follow the Markdown syntax format for layout.
        2. If there are code examples, they must follow standard syntax specifications, have document annotations, and be displayed in code blocks.
        3. The output must be strictly in the specified language, {language}.
        4. Do not have redundant output, including concluding remarks.
        5. Strict requirement not to output the topic "{topic}".
        """

        for _, i in self.node.children.items():
            prompt = CONTENT_PROMPT.format(
                topic=topic, language=self.language, directory=i.key)
            i.set_llm(self.llm) ## 这里要设置一下llm，即使设置为None，也可以正常工作，但不设置就没法正常工作
            ## 为子节点设置context，也就是Prompt，ActionNode中我们将instruction放空，instruction和context都会作为prompt给大模型
            ## 所以两者有一个为空也没关系，只要prompt完整就行
            i.set_context(prompt)
            child = await i.simple_fill(schema="raw", mode="auto") ## 这里的schema注意写"raw"
            self.total_content += child.content ## 组装所有子节点的输出
        logger.info("writecontent:", self.total_content)
        return self.total_content

In [21]:
class TutorialAssistant(Role):

    topic: str = ""
    main_title: str = ""
    total_content: str = ""
    language: str = "Chinese"

    def __init__(
        self,
        name: str = "Stitch",
        profile: str = "Tutorial Assistant",
        goal: str = "Generate tutorial documents",
        constraints: str = "Strictly follow Markdown's syntax, with neat and standardized layout",
        language: str = "Chinese",
    ):
        super().__init__()
        self._init_actions([WriteDirectory(language=language)])
        self.language = language

    async def _think(self) -> None:
        """Determine the next action to be taken by the role."""
        logger.info(self.rc.state)
        # logger.info(self,)
        if self.rc.todo is None:
            self._set_state(0)
            return

        if self.rc.state + 1 < len(self.states):
            self._set_state(self.rc.state + 1)
        else:
            self.rc.todo = None

    async def _handle_directory(self, titles: Dict) -> Message:
        self.main_title = titles.get("title")
        directory = f"{self.main_title}\n"
        self.total_content += f"# {self.main_title}"
        action_nodes = list()
        # actions = list()
        for first_dir in titles.get("directory"):
            logger.info(f"================== {first_dir}")
            action_nodes.append(ActionNode(
                key=f"{first_dir}",
                expected_type=str,
                instruction="",
                example=""))
            key = list(first_dir.keys())[0]
            directory += f"- {key}\n"
            for second_dir in first_dir[key]:
                directory += f"  - {second_dir}\n"

        self._init_actions([WriteContent(language=self.language, action_nodes=action_nodes)])
        self.rc.todo = None
        return Message(content=directory)

    async def _act(self) -> Message:
        """Perform an action as determined by the role.

        Returns:
            A message containing the result of the action.
        """
        todo = self.rc.todo
        if type(todo) is WriteDirectory:
            msg = self.rc.memory.get(k=1)[0]
            self.topic = msg.content
            resp = await todo.run(topic=self.topic)
            logger.info(resp)
            return await self._handle_directory(resp)
        resp = await todo.run(topic=self.topic)
        logger.info(resp)
        if self.total_content != "":
            self.total_content += "\n\n\n"
        self.total_content += resp
        return Message(content=resp, role=self.profile)

    async def _react(self) -> Message:
        """Execute the assistant's think and actions.

        Returns:
            A message containing the final result of the assistant's actions.
        """
        while True:
            await self._think()
            if self.rc.todo is None:
                break
            msg = await self._act()
        root_path = TUTORIAL_PATH / datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        logger.info(f"Write tutorial to {root_path}")
        await File.write(root_path, f"{self.main_title}.md", self.total_content.encode('utf-8'))
        return msg

In [22]:
async def main():
    msg = "强化学习"
    role = TutorialAssistant()
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)

await main()

2024-01-21 06:09:05.076 | INFO     | metagpt.config:get_default_llm_provider_enum:124 - LLMProviderEnum.AZURE_OPENAI Model: gpt35turbo
2024-01-21 06:09:05.078 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.AZURE_OPENAI
2024-01-21 06:09:05.169 | INFO     | metagpt.config:get_default_llm_provider_enum:124 - LLMProviderEnum.AZURE_OPENAI Model: gpt35turbo
2024-01-21 06:09:05.172 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.AZURE_OPENAI
2024-01-21 06:09:05.264 | INFO     | __main__:main:4 - 强化学习
2024-01-21 06:09:05.267 | INFO     | __main__:_think:22 - -1


prompt:  
        The topic of tutorial is 强化学习. Please provide the specific table of contents for this tutorial, strictly following the following requirements:
        1. The output must be strictly in the specified language, Chinese.
        2. Answer strictly in the dictionary format like {"title": "xxx", "directory": [{"dir 1": ["sub dir 1", "sub dir 2"]}, {"dir 2": ["sub dir 3", "sub dir 4"]}]}.
        3. The directory should be as specific and sufficient as possible, with a primary and secondary directory.The secondary directory is in the array.
        4. Do not have extra spaces or line breaks.
        5. Each directory title has practical significance.
        教程的主题是强化学习。请按照以下要求提供本教程的具体目录：
        1. 输出必须严格符合指定语言，Chinese。
        2. 回答必须严格按照字典格式，如{"title": "xxx", "directory": [{"dir 1": ["sub dir 1", "sub dir 2"]}, {"dir 2": ["sub dir 3", "sub dir 4"]}]}。
        3. 目录应尽可能具体和充分，包括一级和二级目录。二级目录在数组中。
        4. 不要有额外的空格或换行符。
        5. 每个目录标题都具有实际意义。
        
{"title": "强化学习教程",

2024-01-21 06:09:11.466 | INFO     | __main__:_act:66 - {'title': '强化学习教程', 'directory': [{'第一章': ['强化学习概述', '强化学习应用领域']}, {'第二章': ['马尔可夫决策过程', '值函数', '策略函数']}, {'第三章': ['Q学习', 'SARSA算法', '深度Q网络']}, {'第四章': ['策略梯度方法', 'Actor-Critic算法']}, {'第五章': ['强化学习中的探索与利用', '多臂赌博机问题', 'ε-贪婪策略']}, {'第六章': ['强化学习中的函数逼近', '线性函数逼近', '神经网络函数逼近']}, {'第七章': ['强化学习中的深度学习', '深度强化学习框架', '深度Q网络的改进']}, {'第八章': ['强化学习中的连续动作空间', '确定性策略梯度', '深度确定性策略梯度']}, {'第九章': ['强化学习中的多智能体系统', '博弈论与强化学习', '多智能体强化学习算法']}, {'第十章': ['强化学习中的实践应用', '机器人控制', '自动驾驶']}, {'第十一章': ['强化学习的未来发展方向', '强化学习的挑战与机遇']}]}
2024-01-21 06:09:11.467 | INFO     | __main__:_handle_directory:40 - ================== {'第一章': ['强化学习概述', '强化学习应用领域']}
2024-01-21 06:09:11.474 | INFO     | __main__:_handle_directory:40 - ================== {'第二章': ['马尔可夫决策过程', '值函数', '策略函数']}
2024-01-21 06:09:11.476 | INFO     | __main__:_handle_directory:40 - ================== {'第三章': ['Q学习', 'SARSA算法', '深度Q网络']}
2024-01-21 06:09:11.477 | INFO     | __main__:_handle_directo

多智能体强化学习算法"]},
    {"第十章": ["强化学习中的实践应用", "机器人控制", "自动驾驶"]},
    {"第十一章": ["强化学习的未来发展方向", "强化学习的挑战与机遇"]}
]}### 第一章: 强化学习概述

强化学习是一种机器学习方法，通过智能体与环境的交互来学习最优的行为策略。在强化学习中，智能体通过观察环境的状态，采取相应的动作，并根据环境的反馈获得奖励或惩罚。智能体的目标是通过与环境的交互，最大化累积奖励。

#### 强化学习的基本要素

强化学习包含以下基本要素：

1. **智能体（Agent）**：智能体是进行学习的主体，它通过观察环境的状态并采取动作来与环境进行交互。

2. **环境（Environment）**：环境是智能体所处的外部环境，它可以是真实世界中的物理环境，也可以是虚拟环境。环境会根据智能体的动作给予反馈，并改变自身的状态。

3. **状态（State）**：状态是环境的一种表示，用于描述环境的特征。智能体通过观察环境的状态来做出决策。

4. **动作（Action）**：动作是智能体在某个状态下可以采取的行为。智能体通过选择合适的动作来影响环境。

5. **奖励（Reward）**：奖励是环境对智能体行为的评价，用于指导智能体的学习。智能体的目标是通过最大化累积奖励来学习最优的行为策略。

#### 强化学习的基本流程

强化学习的基本流程如下：

1. **初始化**：初始化智能体和环境的状态。

2. **观察状态**：智能体观察环境的当前状态。

3. **选择动作**：智能体根据观察到的状态选择一个动作。

4. **执行动作**：智能体执行选择的动作，并与环境进行交互。

5. **获得奖励**：环境根据智能体的动作给予奖励或惩罚。

6. **更新策略**：智能体根据获得的奖励更新自己的策略，以便在类似的状态下做出更好的决策。

7. **重复步骤2-6**：重复观察状态、选择动作、执行动作、获得奖励和更新策略的过程，直到达到某个终止条件。

#### 强化学习的应用领域

强化学习在许多领域都有广泛的应用，包括但不限于以下几个方面：

1. **游戏领域**：强化学习在游戏领域中得到了广泛的应用，如围棋、象棋、扑克等。通过强化学习，智能体可以学习到优秀的游戏策略，甚至超越人

2024-01-21 06:11:19.240 | INFO     | __main__:run:50 - writecontent:
2024-01-21 06:11:19.242 | INFO     | __main__:_act:69 - ### 第一章: 强化学习概述

强化学习是一种机器学习方法，通过智能体与环境的交互来学习最优的行为策略。在强化学习中，智能体通过观察环境的状态，采取相应的动作，并根据环境的反馈获得奖励或惩罚。智能体的目标是通过与环境的交互，最大化累积奖励。

#### 强化学习的基本要素

强化学习包含以下基本要素：

1. **智能体（Agent）**：智能体是进行学习的主体，它通过观察环境的状态并采取动作来与环境进行交互。

2. **环境（Environment）**：环境是智能体所处的外部环境，它可以是真实世界中的物理环境，也可以是虚拟环境。环境会根据智能体的动作给予反馈，并改变自身的状态。

3. **状态（State）**：状态是环境的一种表示，用于描述环境的特征。智能体通过观察环境的状态来做出决策。

4. **动作（Action）**：动作是智能体在某个状态下可以采取的行为。智能体通过选择合适的动作来影响环境。

5. **奖励（Reward）**：奖励是环境对智能体行为的评价，用于指导智能体的学习。智能体的目标是通过最大化累积奖励来学习最优的行为策略。

#### 强化学习的基本流程

强化学习的基本流程如下：

1. **初始化**：初始化智能体和环境的状态。

2. **观察状态**：智能体观察环境的当前状态。

3. **选择动作**：智能体根据观察到的状态选择一个动作。

4. **执行动作**：智能体执行选择的动作，并与环境进行交互。

5. **获得奖励**：环境根据智能体的动作给予奖励或惩罚。

6. **更新策略**：智能体根据获得的奖励更新自己的策略，以便在类似的状态下做出更好的决策。

7. **重复步骤2-6**：重复观察状态、选择动作、执行动作、获得奖励和更新策略的过程，直到达到某个终止条件。

#### 强化学习的应用领域

强化学习在许多领域都有广泛的应用，包括但不限于以下几个方面：

1. **游戏领域**：强化学习在游戏领域中得到了广泛的应用，如围棋、象棋、扑克等。通过强化学习，智能体

化学习。这些方法的发展将进一步推动强化学习在各个领域的应用和性能提升。

## Homework: Novel writor

In [1]:
import asyncio
import re

from metagpt.actions.action import Action, ActionNode
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message
from typing import Dict
from metagpt.utils.common import OutputParser

from metagpt.const import TUTORIAL_PATH
from metagpt.utils.file import File

from datetime import datetime


2024-01-21 09:38:42.596 | INFO     | metagpt.const:get_metagpt_package_root:32 - Package root set to /content
2024-01-21 09:38:42.842 | INFO     | metagpt.config:get_default_llm_provider_enum:124 - LLMProviderEnum.AZURE_OPENAI Model: gpt35turbo
2024-01-21 09:38:42.845 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.AZURE_OPENAI


In [2]:
DIRECTORY_STRUCTION = """
    We need you to weave a captivating science fiction story".
    您现在是一个在技术重塑社会的世界里的富有远见的科幻小说作者。
    我们需要您编织一个引人入胜的科幻故事。```

    """

# 实例化一个ActionNode，输入对应的参数
DIRECTORY_WRITE = ActionNode(
    # ActionNode的名称
    key="DirectoryWrite",
    # 期望输出的格式
    expected_type=str,
    # 命令文本
    instruction=DIRECTORY_STRUCTION,
    # 例子输入，在这里我们可以留空
    example="",
 )

In [7]:


class WriteDirectory(Action):
    """Action class for writing tutorial directories.

    Args:
        name: The name of the action.
        language: The language to output, default is "Chinese".

        用于编写教程目录的动作类。
        参数：
        name：动作的名称。
        language：输出的语言，默认为"Chinese"。
    """

    language: str = "Chinese"

    def __init__(self, name: str = "", language: str = "Chinese", *args, **kwargs):
        super().__init__()
        self.language = language

    async def run(self, topic: str, *args, **kwargs) -> Dict:
        """Execute the action to generate a tutorial directory according to the topic.

        Args:
            topic: The tutorial topic.

        Returns:
            the tutorial directory information, including {"title": "xxx", "directory": [{"dir 1": ["sub dir 1", "sub dir 2"]}]}.
        根据主题执行生成教程目录的操作。
            参数：
            topic：教程主题。
            返回：
            教程目录信息，包括{"title": "xxx", "directory": [{"dir 1": ["sub dir 1", "sub dir 2"]}]}.
        """

        DIRECTORY_PROMPT = """
        The theme of your science fiction story is {topic}. Please provide the detailed outline for this story, adhering strictly to the following guidelines:
        1. The outline must be strictly in the specified language, {language}.
        2. Respond strictly in the structured format like {{"title": "xxx", "directory": [{{"dir 1": ["sub dir 1", "sub dir 2"]}}, {{"dir 2": ["sub dir 3", "sub dir 4"]}}]}}.
        3. The outline should be as detailed and comprehensive as possible, with primary chapters and secondary sections. Secondary sections are in the array.
        4. Do not include extra spaces or line breaks.
        5. Each chapter and section title must be meaningful and relevant to the story.
        您的科幻小说主题是{topic}。请按照以下指南提供这个故事的详细大纲：
        1. 大纲必须严格使用指定语言，{language}。
        2. 回答必须严格按照结构化格式，如{{"title": "xxx", "directory": [{{"dir 1": ["sub dir 1", "sub dir 2"]}}, {{"dir 2": ["sub dir 3", "sub dir 4"]}}]}}。
        3. 大纲应尽可能详细和全面，包括主要章节和次要部分。次要部分在数组中。
        4. 不要包含额外的空格或换行符。
        5. 每个章节和部分标题必须对故事有意义且相关。
        """

        # 我们设置好prompt，作为ActionNode的输入
        prompt = DIRECTORY_PROMPT.format(topic=topic, language=self.language)
        # resp = await self._aask(prompt=prompt)
        # 直接调用ActionNode.fill方法，注意输入llm
        # 该方法会返回self，也就是一个ActionNode对象
        print("prompt: ", prompt)
        resp_node = await DIRECTORY_WRITE.fill(context=prompt, llm=self.llm, schema="raw")
        # 选取ActionNode.content，获得我们期望的返回信息
        resp = resp_node.content
        print(resp)
        return OutputParser.extract_struct(resp, dict)

In [14]:
class WriteContent(Action):
    """Action class for writing tutorial content.

    Args:
        name: The name of the action.
        directory: The content to write.
        language: The language to output, default is "Chinese".
    """

    language: str = "Chinese"
    directory: str = ""
    total_content: str = "" ## 组装所有子节点的输出

    def __init__(self, name: str = "", action_nodes: list = [], language: str = "Chinese", *args, **kwargs):
        super().__init__()
        self.language = language
        self.node = ActionNode.from_children("WRITE_CONTENT_NODES", action_nodes) ## 根据传入的action_nodes列表，生成一个父节点

    async def run(self, topic: str, *args, **kwargs) -> str:
        SCI_FI_WRITER_PROMPT = """
        You are now an imaginative science fiction writer in a world of advanced technology and unknown mysteries.
        We need you to write a science fiction story with the theme "{topic}".
        """
        STORY_CONTENT_PROMPT = SCI_FI_WRITER_PROMPT + """
        Now I will give you the chapter titles for the theme.
        Please output the detailed narrative and plot elements for each title.
        If there are dialogues or descriptions of technology, please provide them according to the narrative style of science fiction.
        Without dialogue or technology description, focus on the story development.

        The chapter titles for the theme are as follows:
        {directory}

        Strictly limit output according to the following requirements:
        1. Follow a structured narrative format suitable for a novel.
        2. If there are dialogues or technology descriptions, they must be vivid, engaging, and fit within the story's world.
        3. The output must be strictly in the specified language, {language}.
        4. Do not have redundant output, including unnecessary descriptions or side plots.
        5. Strict requirement not to deviate from the theme "{topic}".
        """


        for _, i in self.node.children.items():
            prompt = STORY_CONTENT_PROMPT.format(
                topic=topic, language=self.language, directory=i.key)
            # self.llm.temperature = 0.7
            i.set_llm(self.llm)

            i.set_context(prompt)
            child = await i.simple_fill(schema="raw", mode="auto") ## 这里的schema注意写"raw"
            self.total_content += child.content ## 组装所有子节点的输出
        logger.info("writecontent:", self.total_content)
        return self.total_content


In [15]:
class NovelWriter(Role):

    topic: str = ""
    main_title: str = ""
    total_content: str = ""
    language: str = "Chinese"

    def __init__(
        self,
        name: str = "Storysmith",
        profile: str = "Novel Writer",
        goal: str = "Create engaging and imaginative science fiction stories",
        constraints: str = "Adhere to a structured narrative format, with vivid and engaging content",
        language: str = "Chinese",
    ):
        super().__init__()
        self._init_actions([WriteDirectory(language=language)])
        self.language = language

    async def _think(self) -> None:
        """Determine the next action to be taken by the role."""
        logger.info(self.rc.state)
        # logger.info(self,)
        if self.rc.todo is None:
            self._set_state(0)
            return

        if self.rc.state + 1 < len(self.states):
            self._set_state(self.rc.state + 1)
        else:
            self.rc.todo = None

    async def _handle_directory(self, titles: Dict) -> Message:
        self.main_title = titles.get("title")
        directory = f"{self.main_title}\n"
        self.total_content += f"# {self.main_title}"
        action_nodes = list()
        # actions = list()
        for first_dir in titles.get("directory"):
            logger.info(f"================== {first_dir}")
            action_nodes.append(ActionNode(
                key=f"{first_dir}",
                expected_type=str,
                instruction="",
                example=""))
            key = list(first_dir.keys())[0]
            directory += f"- {key}\n"
            for second_dir in first_dir[key]:
                directory += f"  - {second_dir}\n"

        self._init_actions([WriteContent(language=self.language, action_nodes=action_nodes)])
        self.rc.todo = None
        return Message(content=directory)

    async def _act(self) -> Message:
        """Perform an action as determined by the role.

        Returns:
            A message containing the result of the action.
        """
        todo = self.rc.todo
        if type(todo) is WriteDirectory:
            msg = self.rc.memory.get(k=1)[0]
            self.topic = msg.content
            resp = await todo.run(topic=self.topic)
            logger.info(resp)
            return await self._handle_directory(resp)
        resp = await todo.run(topic=self.topic)
        logger.info(resp)
        if self.total_content != "":
            self.total_content += "\n\n\n"
        self.total_content += resp
        return Message(content=resp, role=self.profile)

    async def _react(self) -> Message:
        """Execute the assistant's think and actions.

        Returns:
            A message containing the final result of the assistant's actions.
        """
        while True:
            await self._think()
            if self.rc.todo is None:
                break
            msg = await self._act()
        root_path = TUTORIAL_PATH / datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        logger.info(f"Write tutorial to {root_path}")
        await File.write(root_path, f"{self.main_title}.md", self.total_content.encode('utf-8'))
        return msg

In [16]:
async def main():
    msg = "城市边缘：赛博朋克的未来幻想"
    role = NovelWriter()
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)

await main()

2024-01-21 09:47:43.978 | INFO     | metagpt.config:get_default_llm_provider_enum:124 - LLMProviderEnum.AZURE_OPENAI Model: gpt35turbo
2024-01-21 09:47:43.989 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.AZURE_OPENAI
2024-01-21 09:47:44.275 | INFO     | metagpt.config:get_default_llm_provider_enum:124 - LLMProviderEnum.AZURE_OPENAI Model: gpt35turbo
2024-01-21 09:47:44.280 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.AZURE_OPENAI
2024-01-21 09:47:44.436 | INFO     | __main__:main:4 - 城市边缘：赛博朋克的未来幻想
2024-01-21 09:47:44.442 | INFO     | __main__:_think:22 - -1


prompt:  
        The theme of your science fiction story is 城市边缘：赛博朋克的未来幻想. Please provide the detailed outline for this story, adhering strictly to the following guidelines:
        1. The outline must be strictly in the specified language, Chinese.
        2. Respond strictly in the structured format like {"title": "xxx", "directory": [{"dir 1": ["sub dir 1", "sub dir 2"]}, {"dir 2": ["sub dir 3", "sub dir 4"]}]}.
        3. The outline should be as detailed and comprehensive as possible, with primary chapters and secondary sections. Secondary sections are in the array.
        4. Do not include extra spaces or line breaks.
        5. Each chapter and section title must be meaningful and relevant to the story.
        您的科幻小说主题是城市边缘：赛博朋克的未来幻想。请按照以下指南提供这个故事的详细大纲：
        1. 大纲必须严格使用指定语言，Chinese。
        2. 回答必须严格按照结构化格式，如{"title": "xxx", "directory": [{"dir 1": ["sub dir 1", "sub dir 2"]}, {"dir 2": ["sub dir 3", "sub dir 4"]}]}。
        3. 大纲应尽可能详细和全面，包括主要章节和次要部分。次要部分在数组中。
        4.

2024-01-21 09:47:48.990 | INFO     | __main__:_act:66 - {'title': '城市边缘：赛博朋克的未来幻想', 'directory': [{'第一章': ['引子：城市边缘的黑暗', '节1：主角的背景介绍', '节2：主角的日常生活', '节3：主角的追求']}, {'第二章': ['节1：城市边缘的赛博朋克文化', '节2：主角的赛博朋克启蒙', '节3：主角的技术探索']}, {'第三章': ['节1：城市边缘的黑市交易', '节2：主角的黑市冒险', '节3：主角的黑市交易']}, {'第四章': ['节1：城市边缘的犯罪团伙', '节2：主角的犯罪团伙合作', '节3：主角的犯罪团伙对抗']}, {'第五章': ['节1：城市边缘的科技巨头', '节2：主角的科技巨头合作', '节3：主角的科技巨头对抗']}, {'第六章': ['节1：城市边缘的反抗组织', '节2：主角的反抗组织加入', '节3：主角的反抗组织战斗']}, {'第七章': ['节1：城市边缘的未来展望', '节2：主角的未来选择', '节3：结局：城市边缘的转变']}]}
2024-01-21 09:47:48.992 | INFO     | __main__:_handle_directory:40 - ================== {'第一章': ['引子：城市边缘的黑暗', '节1：主角的背景介绍', '节2：主角的日常生活', '节3：主角的追求']}
2024-01-21 09:47:48.994 | INFO     | __main__:_handle_directory:40 - ================== {'第二章': ['节1：城市边缘的赛博朋克文化', '节2：主角的赛博朋克启蒙', '节3：主角的技术探索']}
2024-01-21 09:47:48.996 | INFO     | __main__:_handle_directory:40 - ================== {'第三章': ['节1：城市边缘的黑市交易', '节2：主角的黑市冒险', '节3：主角的黑市交易']}
2024-01-21 09:47:48.998 | INFO     | __main__:_

斗"
    ]},
    {"第七章": [
        "节1：城市边缘的未来展望",
        "节2：主角的未来选择",
        "节3：结局：城市边缘的转变"
    ]}
]}{"title": "城市边缘：赛博朋克的未来幻想", "directory": [
    {"第一章": [
        "引子：城市边缘的黑暗",
        "节1：主角的背景介绍",
        "节2：主角的日常生活",
        "节3：主角的追求"
    ]},
    {"第二章": [
        "节1：城市边缘的赛博朋克文化",
        "节2：主角的赛博朋克启蒙",
        "节3：主角的技术探索"
    ]},
    {"第三章": [
        "节1：城市边缘的黑市交易",
        "节2：主角的黑市冒险",
        "节3：主角的黑市交易"
    ]},
    {"第四章": [
        "节1：城市边缘的犯罪团伙",
        "节2：主角的犯罪团伙合作",
        "节3：主角的犯罪团伙对抗"
    ]},
    {"第五章": [
        "节1：城市边缘的科技巨头",
        "节2：主角的科技巨头合作",
        "节3：主角的科技巨头对抗"
    ]},
    {"第六章": [
        "节1：城市边缘的反抗组织",
        "节2：主角的反抗组织加入",
        "节3：主角的反抗组织战斗"
    ]},
    {"第七章": [
        "节1：城市边缘的未来展望",
        "节2：主角的未来选择",
        "节3：结局：城市边缘的转变"
    ]}
]}
第一章: 引子：城市边缘的黑暗

节1: 主角的背景介绍

在一个被高楼大厦和闪烁的霓虹灯所包围的城市边缘，生活着一个名叫杰克的年轻人。杰克曾经是一名优秀的工程师，但在一次事故中失去了他的双腿。他被迫离开了他曾经熟悉的高科技公司，被遗忘在这个城市的边缘。

节2: 主角的日常生活

杰克现在过着贫困的生活，住在一个破旧的公寓里。他每天靠着做些零工来维持生计，但他的心中始终充满了对

2024-01-21 09:49:24.902 | INFO     | __main__:run:51 - writecontent:
2024-01-21 09:49:24.904 | INFO     | __main__:_act:69 - 第一章: 引子：城市边缘的黑暗

节1: 主角的背景介绍

在一个被高楼大厦和闪烁的霓虹灯所包围的城市边缘，生活着一个名叫杰克的年轻人。杰克曾经是一名优秀的工程师，但在一次事故中失去了他的双腿。他被迫离开了他曾经熟悉的高科技公司，被遗忘在这个城市的边缘。

节2: 主角的日常生活

杰克现在过着贫困的生活，住在一个破旧的公寓里。他每天靠着做些零工来维持生计，但他的心中始终充满了对科技的渴望。每当他看到城市中那些飞行汽车和智能机器人时，他都感到无比的羡慕和失落。

节3: 主角的追求

杰克决定不再沉溺于自怜之中，他开始研究自己的假肢。他利用废弃的机械零件和电子设备，设计出了一套先进的仿生腿。这套仿生腿不仅能够让他重新行走，还具备了超越常人的速度和力量。

杰克的成就引起了一家科技公司的注意。他们邀请杰克加入他们的团队，为他们开发更先进的人工智能技术。杰克终于有机会重新融入科技世界，实现自己的追求。

然而，随着杰克深入研究人工智能，他逐渐发现了科技背后的黑暗。他发现这家公司正在利用人工智能来控制人们的思想和行为，以达到他们自己的目的。杰克决定揭露这一阴谋，与公司展开一场生死对决。

在这个城市边缘的黑暗中，杰克将用他的智慧和勇气，引领人们走向光明的未来。第二章: 节1：城市边缘的赛博朋克文化

城市边缘的赛博朋克文化是一个充满了刺激和危险的世界。在这个城市的边缘地带，高楼大厦和繁华的街道逐渐消失，取而代之的是摇摇欲坠的建筑和破败的街巷。这里是贫民窟和犯罪集团的根据地，也是赛博朋克文化的发源地。

在这个节1中，我将介绍城市边缘的赛博朋克文化的特点和背景。赛博朋克文化是一种反抗和反体制的文化，它崇尚自由和个人主义。人们在这里追求刺激和自由，他们通过改造自己的身体和大胆的行动来表达自己的个性。

主角是一个年轻的赛博朋克爱好者，他在这个城市边缘长大，深受赛博朋克文化的影响。他热爱电子音乐和极限运动，经常参加地下派对和街头竞技。他渴望逃离贫困和束缚，追寻自由和刺激。

在这个节中，我将描述主角与其他赛博朋克爱好者的交流和互动。

努力让人们重新看到了未来的可能性，让城市边缘成为了一个更美好的地方。

这个故事告诉我们，即使在城市边缘，科技也可以成为人类进步的力量。只要我们坚守自己的原则，努力创造出更好的未来，我们就能够改变命运，让城市边缘焕发新的生机。